<h1>Evidencia 1: Almacen y robots</h1>

<p>Integrantes:
<p><i>Miguel Angel Pedraza Aguilar A01284469</i></p>
<p><i>Eugenio Castro A00830392</i></p>
<p><i>Eduardo Ramón A01384225</i></p>
<p><i>Emiliano Sánchez A00831284</i></p>
<p><i>Sergio Chávez A01284297</i></p>

<p>
    Cada robot está equipado con ruedas omnidireccionales y, por lo tanto, puede conducir en las cuatro direcciones. Pueden recoger cajas en celdas de cuadrícula adyacentes con sus manipuladores, luego llevarlas a otra ubicación e incluso construir pilas de hasta cinco cajas. Todos los robots están equipados con la tecnología de sensores más nueva que les permite recibir datos de sensores de las cuatro celdas adyacentes. Por tanto, es fácil distinguir si un campo está libre, es una pared, contiene una pila de cajas (y cuantas cajas hay en la pila) o está ocupado por otro robot. Los robots también tienen sensores de presión equipados que les indican si llevan una caja en ese momento.
</p>

<p>Realiza la siguiente simulación:</p>
<ul>
    <li>Inicializa las posiciones iniciales de las N cajas. Todas las cajas están a nivel de piso,
es decir, no hay pilas de cajas.</li>
    <li>Todos los agentes empiezan en posición aleatorias vacías.</li>
    <li>Se ejecuta el tiempo máximo establecido.</li>
</ul>

<p>Deberás recopilar la siguiente información durante la ejecución:</p>
<ul>
    <li>Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.</li>
    <li>Número de movimientos realizados por todos los robots.</li>
    <li>Analiza si existe una estrategia que podría disminuir el tiempo dedicado, así como
la cantidad de movimientos realizados. ¿Cómo sería? Descríbela.</li>
</ul>

<h2>Imports</h2>
<p>Realizamos los imports de mesa, matplot, numpy, pandas y random necesarios.</p>
<ul>
    <li>Debemos importar de mesa Agent y Model para poder utilizarlos 
</ul>

In [1]:
# Imports

# Importamos de mesa Agent y Model
from mesa import Agent, Model 

# Importamos nuestro grid de manera Multigrid que permite que puedan existir multiples agentes en una celda
from mesa.space import MultiGrid 

# Definimos el tiempo en el que se realizan los pasos, en este caso SimultaneousActivation hace que en cada paso todas las aspiradoras se tienen que mover
from mesa.time import SimultaneousActivation

# Traemos una manera de poder recolectar información con el data collector
from mesa.datacollection import DataCollector

# Imporamos utilidades de matplot
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos numpy, pandas y random para facilitar operaciones
import numpy as np
import pandas as pd
import random

# Importamos el tiempo para poder determinar como funciona el programa
import time
import datetime

<h2>Creamos el modelo</h2>
<p>En este caso decidimos utilizar 3 agentes y 1 modelo:</p>
<ul>
    <li>El agente Robot</li>
    <li>El agente Estantería</li>
    <li>El agente Piso</li>
    <li>El modelo Almacen</li>
</ul>
<p>En este seguimos una estrategia aleatoria donde dentro de nuestro modelo almacen se tienen las cajas y posiciones de nuestros 5 robots de manera aleatoria y los robots se moveran de manera aleatoria a dentro del almacen sin chocar con los estantes que estan designados en posiciones fijas dentro del almacen.</p>
<p>En este caso los robots se irán moviendo dentro del almacén y una vez se encuentren con alguna caja procederan a irse a la estantería corresponiente dependiendo de si dicha estantería tiene menos de 5 cajas de lo contrario los robots se iran desplazando la siguiente estantería y seguiran buscando por el resto de cajas.</p>

In [2]:
# Indice global para saber que estanteria es la que esta disponible en este momento para todos los robots
I = 0

# Funcion para poder determinar los colores en la animacion y asignar cuales son aspiradoras en la habitacion
def obtener_almacen(modelo):
    almacen = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:            
            if isinstance(contenido, Robot):
                almacen[x][y] = 2
            elif isinstance(contenido, Estanteria):
                almacen[x][y] = contenido.estado
            else:
                almacen[x][y] = contenido.estado
                
    return almacen

# Clase del agente Robot
class Robot(Agent):    
    def __init__(self, id_unico, modelo, estanterias_pos):
        super().__init__(id_unico, modelo)
        self.nueva_posicion = None
        self.estanterias_pos = estanterias_pos
        self.movimientos = 0 
        self.est_index = 0
    
    def step(self):
        # Recibimos la variable global que es nuestro indice dentro de las posiciones de estaterias
        global I
        
        # Revisamos los vecinos alrededor nuestro y los almacenamos en una lista
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = True,
            include_center = True
        )      
    
        # Definimos el siguiente estado que va a tener el piso despues de revisar su estado el cual es que ya no tenga una caja
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                # En caso de que encontremos que el piso tiene de estado CAJA, lo que representa que hay una caja en el piso entonces
                # la cambiamos a estado VACIO 
                vecino.siguiente_estado = vecino.estado
                if vecino.siguiente_estado == vecino.CAJA:
                    vecino.siguiente_estado = vecino.VACIO
                    
                    # Ahora nos debemos mover a una posicion frente a la estanteria con el indice actual
                    posFrenteEstante = (self.estanterias_pos[I][0] + 1, self.estanterias_pos[I][1])                        
                    self.nueva_posicion = posFrenteEstante
                    
                    # Una vez estando en esta posicion obtenemos una lista de los vecinos para determinar donde se encuentra la estanteria
                    estanterias = self.model.grid.get_neighbors(
                        self.nueva_posicion,
                        moore = True,
                        include_center = False
                    )                                        
                    
                    # Recorremos la lista
                    for estanteria in estanterias:  
                        # En caso de encontrar una instancia de la estanteria donde la posicion coincida con la estantería que estamos apilando 
                        # en este momento y la cantidad de cajas que tenga sea menor a 5
                        if isinstance(estanteria, Estanteria) and estanteria.pos == self.estanterias_pos[I] and estanteria.cajas < 5:                             
                            # De ser así procedemos a sumar la cantidad de cajas de dicha estanteria
                            estanteria.cajas += 1; 
                            
                            # Además de agregar el estado de la estanteria a 1 para que en la graficacion se vea que se ha dejado la caja ahi
                            estanteria.estado = 1
                            
                            # Revisamos si la cantidad de cajas despues de dejar la ultima es igual a 5 y de ser así aumentamos
                            # el indice global para que todos los robots sepan que ahora deben ir a la siguiente estanteria
                            if estanteria.cajas == 5:
                                I += 1                                
                            break
                            
                    # Disminuimos el numero de cajas
                    self.model.nCajas -= 1
                else:
                    # En caso de que el piso este vacio sin ninguna caja obtenemos las posicion de los vecinos sin tomar en cuenta la que estabamos
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore = True,
                        include_center = False
                    )
                    
                    # Asignamos una nueva posicion random dentro de nuestro vecindario y la asignamos a nueva posicion
                    nueva_posicion = self.random.choice(vecindario)
                    self.nueva_posicion = nueva_posicion
                    
                    # En casa de que la posicon obtenida este en nuestro arreglo de estanterias la cambiaremos por otra posicion
                    while (nueva_posicion in self.estanterias_pos):
                        nueva_posicion = self.random.choice(vecindario)
                        self.nueva_posicion = nueva_posicion                          
                break        
    
    def advance(self):
        # Actualizamos el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore = False,
            include_center = True
        )
        
        # Actualizmaos el estado del piso sea VACIO o con CAJA que se realizo en el step
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.estado = vecino.siguiente_estado
                break
                
        # Si la posicion no es igual a la nueva posicion entonces significa que nos movimos por lo cual aumentamos el numero de movimientos
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            
        # Movemos la aspiradora a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)

# Clase Estanteria que es nuestro segundo agente
class Estanteria(Agent):    
    def __init__(self, unique_id, modelo, pos, cajas=0):
        super().__init__(unique_id, modelo) # Datos iniciales que se usan para el Agent
        self.pos = pos
        self.cajas = cajas        
        self.estado = 3 # El estado con valor de 3 solamente se usa para el color negro para diferenciar que son estanterias
    
# Clase Piso que es nuestro tercer y ultimo agente
class Piso(Agent):
    # Estados posibles de nuestro piso que nos ayuda tambien a la hora de colorear la animacion
    CAJA = 1
    VACIO = 0
    
    def __init__(self, pos, model, estado=VACIO):
        super().__init__(pos, model)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None

# Clase habitacion que representa nuestro modelo
class Almacen(Model):        
    def __init__(self, nCajas):
        self.num_agentes = 5
        self.nCajas = nCajas
        self.grid = MultiGrid(10, 10, True)
        self.schedule = SimultaneousActivation(self)                                
        
        # Posiciones de nuestras 10 estanterias disponibles
        estanterias_pos = [(0,0), (0,1), (0,2), (0,3), (0,4), (0,5), (0,6), (0,7), (0,8), (0,9)]
        
        # Ahora posicionamos las estanterias en cada posicion del arreglo
        for i in range(len(estanterias_pos)):
            estanteria = Estanteria(i, self, estanterias_pos[i])
            self.grid.place_agent(estanteria, estanterias_pos[i])
            self.schedule.add(estanteria)
        
        # Inicializamos lista de celdas vacias
        lista_celdas_vacias = list(self.grid.empties)
        
        # Posicionamos las cajas de forma aleatoria en el almacen
        for celdas in range(self.nCajas):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Piso(celda_vacia, self)
            piso.estado = piso.CAJA
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
        
        # Posicionamos celdas del piso vacias (sin cajas) 
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)
        
        # Posicionar agentes robots 
        for i in range(self.num_agentes):
            celda_vacia = random.choice(lista_celdas_vacias)
            robot = Robot(i + 10, self, estanterias_pos)
            self.grid.place_agent(robot, celda_vacia)
            self.schedule.add(robot)
            lista_celdas_vacias.remove(celda_vacia)
            
        # Usamos data collector para almacenar informacion que se nos pidió
        self.colectordatos = DataCollector(
            # Definimos colectores a nivel de modelo para ver toda la habitacion y a nivel de agente para ver cada movimiento de los agentes
            model_reporters = {'Habitacion' : obtener_almacen},
            agent_reporters = {'Movimientos' : lambda a: getattr(a, 'movimientos', None)}
        )
        
    def step(self):
        # Cada step hacemos que se colecten los datos definidos
        self.colectordatos.collect(self)
        self.schedule.step()
    
    # Revisar si todas las cajas han sido apiladas para terminar de lo contrario seguir corriendo el programa
    def todasCajasApiladas(self):
        if self.nCajas == 0:
            return True
        else:
            return False

In [3]:
# Numero de cajas que habra dispersas en el almacen
nCajas = 12

# Tiempo maximo de ejercución (segundos)
tMax = 0.06

# Realizamos proceso para determinar nuestro tiempo e inicializamos el modelo
tiempo_inicio = str(datetime.timedelta(seconds = tMax))
start_time = time.time()
modelo = Almacen(nCajas)

# Ciclo donde se seguira corriendo el programa hasta que termine de apilar todas las cajas o se termine el tiempo maximo
while((time.time() - start_time) < tMax and not modelo.todasCajasApiladas()):
    modelo.step()
    
# Guardamos el tiempo que le tomó correr al modelo
tiempo_ejecucion = str(datetime.timedelta(seconds = (time.time() - start_time)))

<h2>Visualización</h2>
<p>Usamos matplot con escala de grises para representar nuestro almacen donde:</p>
<ul>
    <li>El color negro representa las estanterias</li>
    <li>El color gris oscuro los robots</li>
    <li>El color gris claro las cajas</li>
</ul>

In [4]:
todo_almacen = modelo.colectordatos.get_model_vars_dataframe()

In [5]:
%%capture

fig, axs = plt.subplots(figsize = (7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todo_almacen.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(todo_almacen.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(todo_almacen))

In [6]:
anim

<h2>Informe</h2>

In [7]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que se apilaran todas las cajas: ', tiempo_ejecucion,'/', tiempo_inicio)
print('Número de movimientos realizados por todos los robots: ', movimientos.tail()['Movimientos'].sum())

Tiempo necesario hasta que se apilaran todas las cajas:  0:00:00.059847 / 0:00:00.060000
Número de movimientos realizados por todos los robots:  750.0


<h3>Primera prueba</h3>
<p>En nuestra segunda prueba corrimos el programa 5 veces y obtuvimos estos datos en promedio:</p>
<ul>
    <li>Se utilizaron <b>5 Robots</b></li>
    <li>Se tenían <b>5 cajas</b> dentro del almacén</li>
    <li>El número <b>promedio de movmientos</b> realizados fueron <b>302 movimientos</b></li>
    <li>Se tenía un <b>tiempo máximo</b> de <b>0.06 segundos</b> pero <b>se completo en promedio</b> en <b>0.027187 segundos</b></li>
</ul>

<p>En este caso de las 5 veces que se ejecutó el programa todas finalizaron de apilar las cajas antes de su tiempo máximo permitido<p>

<h3>Segunda prueba</h3>
<p>En nuestra primera prueba corrimos el programa 5 veces y obtuvimos estos datos en promedio:</p>
<ul>
    <li>Se utilizaron <b>5 Robots</b></li>
    <li>Se tenían <b>14 cajas</b> dentro del almacén</li>
    <li>El número <b>promedio de movmientos</b> realizados fueron <b>612.6 movimientos</b></li>
    <li>Se tenía un <b>tiempo máximo</b> de <b>0.06 segundos</b> pero <b>se completo en promedio</b> en <b>0.051727 segundos</b></li>
</ul>

<p>En este caso de las 5 veces que se ejecutó el programa finalizó 1 vez antes de apilar todas las cajas debido al tiempo máximo y el las otras 4 veces finalizo antes del tiempo máximo establecido<p>

<h3>Tercera prueba</h3>
<p>En nuestra tercera prueba corrimos el programa 5 veces y obtuvimos estos datos en promedio:</p>
<ul>
    <li>Se utilizaron <b>5 Robots</b></li>
    <li>Se tenían <b>30 cajas</b> dentro del almacén</li>
    <li>El número <b>promedio de movmientos</b> realizados fueron <b>707 movimientos</b></li>
    <li>Se tenía un <b>tiempo máximo</b> de <b>0.06 segundos</b> pero <b>se completo en promedio</b> en <b>0.059008 segundos</b></li>
</ul>

<p>En este caso de las 5 veces que se ejecutó el programa finalizó 3 veces antes de apilar todas las cajas debido al tiempo máximo y solamente 2 veces pudo terminar de apilar antes de que terminara el tiempo máximo establecido<p>

<h3>Conclusión</h3>
<p>
    En este caso, al seguir un proceso aleatorio para la seleccion de como se moverían los agentes (robots) a la hora de buscar y llevar las cajas a los estntes podemos apreciar claramente que el número de agentes es proporcional al numero de movimientos realizados y el tiempo en que se pudo terminar o no de apilar todas las cajas antes de que se terminara el tiempo máximo establecido. Esto se comprobó utilizando como siempre 5 robots e ibamos haciendo pruebas con la cantidad de cajas que estaban primero de 5 cajas, después 14 y finalmente 30 cajas. Y pudimos obtener un promedio donde vimos como siguiendo esta técnica aleatoria mientras más cajas haya tendremos mayor cantidad de movimientos y a su vez mayor probabilidad de no terminar antes del tiempo esperado.  
</p>
<p>
    Al ver estos resultados definitivamente creemos que existe alguna mejor estrategia para poder disminuir el tiempo de apilamiento de las cajas y a su vez la cantidad de movimientos realizados. Una de las alternativas que pensamos sería que al siempre tener 5 robots en este caso particular podriamos hacer que revise cada una una fila designada y al finalizar los 5 robots se desplacen otras 5 filas. Con esto podríamos tener siempre un flujo constante y en el mejor de los casos sería bastante rápido de encontrar.
</p>

<p>Otra posible solución algo parecida es designar zonas donde cada robot se puede mover solamente en las zonas especificadas, de esta forma se podría mejorar el tiempo de busqueda ya que limitas las posiciones a las que un robot puede ir entonces puedes visitar cada celda de tu zona más rápido y por ende encontrar todas las cajas de tu zona con. mayor velocidad.
</p>